## Monte Carlo Integration

There are many integration techniques that can be used to compute integrals. But the vast majority of integrals do not have analytic answers! So we must resort to numerical methods of computing integrals.

When it comes to the numerical computation of integrals, there are several options:
- quadrature (eg: midpoint, trapezoidal, Simpson's rule)
- adaptive methods
- Newton-Cotes formulae.

But these options suffer from the curse of dimensionality. As the number of dimensions of the integral increases, you need in general, an exponential amount of function evaluations to compute the integral numerically.

So we consider Monte Carlo integration, which approximates the following integral as
$$\int_{[0, 1]^d} f(x)dx \approx \frac{1}{N} \sum_{i = 1}^{N} f(x_i)$$
where the domain of consideration is a d-dimensional hypercube $[0, 1]^d$, $x_1, x_2,..., x_N \sim U([0, 1]^d)$ are points uniformly sampled from the d-dimensional hypercube, and $N$ is the number of points that we sample.

Note that Monte Carlo integration has a convergence rate that is independent of the number of dimensions of the integral.


One of the classic examples of Monte Carlo integration is the use of Monte Carlo integration to approximate $\pi$.

Consider the function 
$$f(x, y) =  \left\{
\begin{array}{ll}
      1 & x^2 + y^2 <= 1 \\
      0 & else \\
\end{array} 
\right.$$
and the domain $\Omega = [-1, 1] \times [-1, 1]$. 

We note that the integral
$$I = \int_{\Omega} f(x, y)dxdy = \pi $$
and thus by randomly (but crucially, uniformly) sampling $N$ points on $\Omega$, we can approximate $\pi$ by computing the ratio between the number of points within the circle and the total number of sampled points. More precisely, this ratio approximates the area ratio of the circle and the rectangle

In mathematical terms,
$$I_{N} = \frac{1}{N} \sum_{i = 1}^{N} f(x_{i}, y_{i}) = \frac{\pi}{4} $$

In fact, this method can be generalized to estimate the area ratio of arbitrary shapes to a rectangle that inscribes the shape.


In [2]:
from numpy import random
import numpy as np
import math

n_batches = 10
n_samples = 100000
count = 0
radius = 1

for batch in range(n_batches):

    for i in range(n_samples):
        x_i = random.uniform(-1, 1)
        y_i = random.uniform(-1, 1)

        if x_i**2 + y_i**2 <= radius:
            count += 1

print(4*(1/(n_batches*n_samples))*count)
print(math.pi)

3.142356
3.141592653589793


Now we try the integration of some arbitrary functions. We consider the one-dimensional integral
$$ I_1 = \int_{0}^{2} cosh(x) dx$$


In [3]:
n_samples = 20000000
a, b = (0, 2)
rand_x = random.uniform(a, b, n_samples)
y = np.cosh(rand_x)
integral_approx = ((b - a)/n_samples)*y.sum()
print(integral_approx)


3.6266539150440127


Now we try using Monte Carlo integration to compute the value of this particular 3-dimensional integral
$$I = \frac{1}{(2\pi)^3}\int_{-\pi}^{\pi}dx\int_{-\pi}^{\pi}dy\int_{-\pi}^{\pi}dz \frac{1}{1 - \cos(x)\cos(y)\cos(z)}$$

This example follows an example in the GNU Scientific Library and is a particularly relevant example for the theory of random walks - it computes the mean time spent at the origin by a random walk on a body-centered cubic lattice in 3D.

In [14]:
def f(x, y, z):
    return 1/(((2*math.pi)**3)*(1 - np.cos(x)*np.cos(y)*np.cos(z)))

n_samples = 1000000
x_a, x_b = -math.pi, math.pi
y_a, y_b = -math.pi, math.pi
z_a, z_b = -math.pi, math.pi

rand_x = random.uniform(x_a, x_b, n_samples)
rand_y = random.uniform(y_a, y_b, n_samples)
rand_z = random.uniform(z_a, z_b, n_samples)

integral_approx = 0

for i in range(n_samples):
    x = rand_x[i]
    y = rand_y[i]
    z = rand_z[i]

    integral_approx += f(x, y, z)

integral_approx = (((x_b - x_a)*(y_b - y_a)*(z_b - z_a))/n_samples)*integral_approx
integral_approx

1.3986161246540831

Now we try the integration of multidimensional functions on some complicated integration volume. For multidimensional Monte Carlo integration over arbitrary integration volumes, the approximation to such an integral is
$$ \int_{\Omega} f(\vec{x}) dV \approx \frac{V}{N} \sum_{i = 1}^{N}f(\vec{x_i})$$
where $\Omega$ is the domain of integration, $V$ is the volume of integration $N$ is the number of points being sampled.

The example we execute below follows an example from scikit-monaco, a Python package that performs Monte Carlo integration. The function being integrated is $f(x, y) = y^2$ and the integration volume of consideration is ![](rings.png).

In [11]:
def f(x, y):
    return y**2

def g(x, y):
    r = np.sqrt(x**2 + y**2)
    if r >= 2 and r <= 3 and x >= 1 and y >= -2:
        return f(x, y)
    else:
        return 0
    
n_samples = 1000000
x_a, x_b = 1, 3
y_a, y_b = -2, 3

rand_x = random.uniform(x_a, x_b, n_samples)
rand_y = random.uniform(y_a, y_b, n_samples)

integral_approx = 0

for i in range(n_samples):
    x = rand_x[i]
    y = rand_y[i]

    integral_approx += g(x, y)

integral_approx = (((x_b - x_a)*(y_b - y_a))/n_samples)*integral_approx
integral_approx

9.966235974317216

Note that in every example so far, we've performed Monte Carlo integration that is based on uniform sampling of the domain of integration. However, instead of uniformly sampling the domain of integration, we can use arbitrary probability distributions to sample the domain of integration. This is particularly relevant for importance sampling, which we will cover in Monte Carlo estimation. The use of arbitrary probability distributions is also relevant in variance reduction of Monte Carlo techniques.


There are also other more advanced Monte Carlo integration techniques, for example the MISER algorithm, which is based on recursive stratified sampling. There is also the VEGAS algorithm, which combines importance sampling and stratified sampling.

